In [1]:
import keras
from keras.layers import LSTM, GRU, Dense, GlobalMaxPool1D, Embedding, Dropout
from keras.models import Sequential
from keras.preprocessing import text, sequence
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
!matplotlib inline
np.random.seed(0)

Using TensorFlow backend.
'matplotlib' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
episodes=pd.read_csv('episodes_cleaned.csv',index_col='Unnamed: 0')
scripts=pd.read_csv('scripts_cleaned.csv',index_col='Unnamed: 0')

In [35]:
gek_dial_df=scripts[(scripts['character_stripped']=='george')|(scripts['character_stripped']=='kramer')|(scripts['character_stripped']=='elaine')]
gek_dial_df=gek_dial_df[['Dialogue','character_stripped']]
gek_dial_df.tail()

,Dialogue,character_stripped
54585,cause you only get one call. the prison call i...,elaine
54587,"i got it - it's out! how about that, huh? oh, ...",kramer
54589,really?,george
54591,haven't we had this conversation before?,george
54593,i think we have.,george


In [36]:
dummies=pd.get_dummies(gek_dial_df.character_stripped)

In [95]:
labels

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [96]:
labels=dummies.values
labels

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0]], dtype=uint8)

In [73]:
tokenizer = text.Tokenizer(num_words=14000)
tokenizer.fit_on_texts(list(gek_dial_df.Dialogue.values))
list_tokenized_train = tokenizer.texts_to_sequences(gek_dial_df.Dialogue.values)
X_t = sequence.pad_sequences(list_tokenized_train,maxlen=100)
X_t

array([[   0,    0,    0, ...,   42,    2,  266],
       [   0,    0,    0, ..., 2289,    3, 2070],
       [   0,    0,    0, ...,   46,   14,  100],
       ...,
       [   0,    0,    0, ...,    0,    0,   68],
       [   0,    0,    0, ...,   14,  607,  280],
       [   0,    0,    0, ...,   56,   48,   29]])

In [105]:
lstm_model = Sequential()
lstm_model.add(Embedding(14000,128))
lstm_model.add(LSTM(50, return_sequences=True))
lstm_model.add(GlobalMaxPool1D())
lstm_model.add(Dense(512 ))
lstm_model.add(Activation('relu'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(256 ))
lstm_model.add(Activation('relu'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(128, ))
lstm_model.add(Activation('relu'))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(3, activation='softmax'))

In [106]:
lstm_model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [107]:
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, None, 128)         1792000   
_________________________________________________________________
lstm_11 (LSTM)               (None, None, 50)          35800     
_________________________________________________________________
global_max_pooling1d_11 (Glo (None, 50)                0         
_________________________________________________________________
dense_55 (Dense)             (None, 512)               26112     
_________________________________________________________________
activation_40 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_44 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_56 (Dense)             (None, 256)               131328    
__________

In [111]:
lstm_history=lstm_model.fit(X_t,
                labels,
                epochs=60,
                batch_size=256,
                validation_split=0.1)

Train on 22134 samples, validate on 2460 samples
Epoch 1/60
22134/22134 [==============================] - 17s 786us/step - loss: 0.7518 - acc: 0.6575 - val_loss: 1.2062 - val_acc: 0.4573
Epoch 2/60
22134/22134 [==============================] - 17s 747us/step - loss: 0.6774 - acc: 0.6946 - val_loss: 1.3166 - val_acc: 0.4508
Epoch 3/60
22134/22134 [==============================] - 17s 777us/step - loss: 0.6154 - acc: 0.7246 - val_loss: 1.4479 - val_acc: 0.4439
Epoch 4/60
22134/22134 [==============================] - 18s 819us/step - loss: 0.5586 - acc: 0.7465 - val_loss: 1.6756 - val_acc: 0.4431
Epoch 5/60
22134/22134 [==============================] - 18s 819us/step - loss: 0.5093 - acc: 0.7643 - val_loss: 1.8373 - val_acc: 0.4419
Epoch 6/60
22134/22134 [==============================] - 19s 870us/step - loss: 0.4720 - acc: 0.7821 - val_loss: 1.9939 - val_acc: 0.4366
Epoch 7/60
22134/22134 [==============================] - 18s 803us/step - loss: 0.4379 - acc: 0.7980 - val_loss: 2.1

In [109]:
from keras.layers import Activation 
num_words=14000
num_classes = 2

model = Sequential()
model.add(Embedding(14000,128))
model.add(Dense(512 ))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256 ))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128, ))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_t, labels, epochs=60, batch_size=256,validation_split=0.1)

ValueError: Error when checking target: expected activation_46 to have 3 dimensions, but got array with shape (24594, 3)

In [110]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, None, 128)         1792000   
_________________________________________________________________
dense_59 (Dense)             (None, None, 512)         66048     
_________________________________________________________________
activation_43 (Activation)   (None, None, 512)         0         
_________________________________________________________________
dropout_47 (Dropout)         (None, None, 512)         0         
_________________________________________________________________
dense_60 (Dense)             (None, None, 256)         131328    
_________________________________________________________________
activation_44 (Activation)   (None, None, 256)         0         
_________________________________________________________________
dropout_48 (Dropout)         (None, None, 256)         0         
__________

In [92]:
X_t.shape

(24594, 100)